In [3]:
# Installing necessary libraries (only if you haven't installed them)
# !pip install pandas joblib

import pandas as pd
import json
import random
import joblib # This is to load your saved ML models from Milestone 2
import os

print("Libraries imported successfully!")

Libraries imported successfully!


In [4]:
# This is our database of food recommendations
diet_db = {
    "Diabetes": {
        "Recommended": ["Whole Grains", "Leafy Greens", "Bitter Gourd"],
        "Avoid": ["Sugar", "White Rice", "Potatoes"],
        "Breakfast": ["Oats with walnuts", "Vegetable Dalia", "Boiled Egg Whites"],
        "Lunch": ["Brown rice with Spinach Dal", "Quinoa Salad", "Grilled Tofu"],
        "Dinner": ["Lentil soup", "Baked fish with broccoli", "Paneer stir-fry"]
    },
    "High_BP": {
        "Recommended": ["Bananas", "Yogurt", "Garlic", "Flaxseeds"],
        "Avoid": ["Extra Salt", "Pickles", "Processed Meat"],
        "Breakfast": ["Banana Smoothie", "Unsalted Porridge", "Low-fat Yogurt"],
        "Lunch": ["Curd Rice (No Salt)", "Boiled Chicken with greens", "Veg Soup"],
        "Dinner": ["Bottle Gourd Soup", "Steamed fish", "Mixed Bean Salad"]
    },
    "Anemia": {
        "Recommended": ["Spinach", "Beetroot", "Dates", "Jaggery"],
        "Avoid": ["Excess Coffee", "Tea during meals"],
        "Breakfast": ["Spinach omelet", "Beetroot Smoothie", "Dates bowl"],
        "Lunch": ["Chickpea Salad", "Mutton/Chicken Liver", "Rajma with brown rice"],
        "Dinner": ["Moringa soup", "Lentil stew", "Steamed broccoli"]
    },
    "Obesity": {
        "Recommended": ["High Fiber Foods", "Protein", "Warm Water"],
        "Avoid": ["Fried Food", "Mayo", "Sugary Drinks"],
        "Breakfast": ["Sprouted Moong Salad", "Boiled Egg Whites", "Green tea"],
        "Lunch": ["Cabbage stir-fry", "Lentil Soup", "Large Green Salad"],
        "Dinner": ["Clear vegetable broth", "Paneer with cucumber", "Mixed bean soup"]
    }
}

In [5]:
def generate_personalized_diet(medical_condition, extra_rules):
    """
    medical_condition: The string result from Milestone 2 (e.g., 'Diabetes')
    extra_rules: The list of rules from Milestone 3 (e.g., ['Low salt'])
    """
    # 1. Fetch the base plan from our database
    # If the condition isn't found, it gives a "General Healthy" default
    base = diet_db.get(medical_condition, {
        "Recommended": ["Fruits", "Vegetables"], 
        "Avoid": ["Junk Food"],
        "Breakfast": ["Fruit Bowl"], "Lunch": ["Salad"], "Dinner": ["Soup"]
    })
    
    # 2. Pick one random dish for each meal for variety
    plan = {
        "Report Summary": f"Personalized Plan for {medical_condition}",
        "Doctor's Notes Integrated": extra_rules,
        "Daily Menu": {
            "Breakfast": random.choice(base["Breakfast"]),
            "Lunch": random.choice(base["Lunch"]),
            "Dinner": random.choice(base["Dinner"])
        },
        "What to Eat More": base["Recommended"],
        "What to Stop Eating": list(set(base["Avoid"] + extra_rules)) # Combines both
    }
    
    return plan

In [6]:
# Simulating the inputs from previous milestones
final_plan = generate_personalized_diet("Diabetes", ["Low salt diet", "No oily food"])

# Display the final result nicely
print(json.dumps(final_plan, indent=4))

{
    "Report Summary": "Personalized Plan for Diabetes",
    "Doctor's Notes Integrated": [
        "Low salt diet",
        "No oily food"
    ],
    "Daily Menu": {
        "Breakfast": "Vegetable Dalia",
        "Lunch": "Grilled Tofu",
        "Dinner": "Paneer stir-fry"
    },
    "What to Eat More": [
        "Whole Grains",
        "Leafy Greens",
        "Bitter Gourd"
    ],
    "What to Stop Eating": [
        "No oily food",
        "White Rice",
        "Sugar",
        "Low salt diet",
        "Potatoes"
    ]
}


In [7]:
# Load the model you trained in Milestone 2
try:
    # Adjust the path if your 'models' folder is in a different place
    model = joblib.load('../models/best_model.pkl')
    scaler = joblib.load('../models/scaler.pkl')
    print("Success: Milestone 2 Model and Scaler loaded!")
except:
    print("Note: Model files not found. We will use manual input for now.")

Success: Milestone 2 Model and Scaler loaded!


In [8]:
def predict_condition_from_numbers(numeric_data):
    """
    numeric_data: A list of values like [Glucose, BloodPressure, BMI]
    """
    # 1. Scale the data (just like you did in Milestone 2)
    scaled_data = scaler.transform([numeric_data])
    
    # 2. Get the prediction (0 or 1)
    prediction = model.predict(scaled_data)[0]
    
    # 3. Map the number to a name
    if prediction == 1:
        return "Diabetes"
    else:
        return "General_Healthy"

# Example: Testing with sample numbers
# test_result = predict_condition_from_numbers([150, 85, 30])
# print(f"AI Prediction: {test_result}")

In [10]:
# --- FIXED FINAL INTEGRATION ---

# 1. Simulating Data from Milestone 1 (OCR)
# IMPORTANT: You must provide all 6 values that your model expects!
# [Glucose, BloodPressure, SkinThickness, Insulin, BMI, Age]
patient_metrics = [140, 85, 20, 79, 31.2, 45] 

# 2. Getting Prediction from Milestone 2 (ML)
# This will now work because we are passing 6 features!
detected_disease = predict_condition_from_numbers(patient_metrics)

# 3. Simulating Rules from Milestone 3 (NLP)
extracted_doctor_rules = ["Avoid salt", "Drink more water"]

# 4. Final Output from Milestone 4
final_report = generate_personalized_diet(detected_disease, extracted_doctor_rules)

print("===== AI-NUTRICARE FINAL OUTPUT =====")
import pprint
pprint.pprint(final_report)

===== AI-NUTRICARE FINAL OUTPUT =====
{'Daily Menu': {'Breakfast': 'Fruit Bowl', 'Dinner': 'Soup', 'Lunch': 'Salad'},
 "Doctor's Notes Integrated": ['Avoid salt', 'Drink more water'],
 'Report Summary': 'Personalized Plan for General_Healthy',
 'What to Eat More': ['Fruits', 'Vegetables'],
 'What to Stop Eating': ['Drink more water', 'Avoid salt', 'Junk Food']}


c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


In [11]:
import pandas as pd

def generate_7_day_table(condition):
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    week_plan = []
    
    for day in days:
        meals = generate_personalized_diet(condition, [])["Daily Menu"]
        week_plan.append({"Day": day, "Breakfast": meals["Breakfast"], "Lunch": meals["Lunch"], "Dinner": meals["Dinner"]})
    
    return pd.DataFrame(week_plan)

# Display the 7-day plan
weekly_table = generate_7_day_table(detected_disease)
from IPython.display import display
display(weekly_table)

,Day,Breakfast,Lunch,Dinner
0,Monday,Fruit Bowl,Salad,Soup
1,Tuesday,Fruit Bowl,Salad,Soup
2,Wednesday,Fruit Bowl,Salad,Soup
3,Thursday,Fruit Bowl,Salad,Soup
4,Friday,Fruit Bowl,Salad,Soup
5,Saturday,Fruit Bowl,Salad,Soup
6,Sunday,Fruit Bowl,Salad,Soup


In [12]:
import pandas as pd
from IPython.display import display

def generate_weekly_report(condition, extra_rules):
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    weekly_data = []
    
    for day in days:
        # Get a random meal set for each day
        daily_plan = generate_personalized_diet(condition, extra_rules)
        meals = daily_plan["Daily Menu"]
        
        weekly_data.append({
            "Day": day,
            "Breakfast": meals["Breakfast"],
            "Lunch": meals["Lunch"],
            "Dinner": meals["Dinner"]
        })
    
    # Create a DataFrame to make it look like a professional table
    df_weekly = pd.DataFrame(weekly_data)
    return df_weekly

# Let's run it!
weekly_df = generate_weekly_report(detected_disease, extracted_doctor_rules)

print(f"\n7-DAY DIET PLAN FOR: {detected_disease}")
display(weekly_df)


7-DAY DIET PLAN FOR: General_Healthy


,Day,Breakfast,Lunch,Dinner
0,Monday,Fruit Bowl,Salad,Soup
1,Tuesday,Fruit Bowl,Salad,Soup
2,Wednesday,Fruit Bowl,Salad,Soup
3,Thursday,Fruit Bowl,Salad,Soup
4,Friday,Fruit Bowl,Salad,Soup
5,Saturday,Fruit Bowl,Salad,Soup
6,Sunday,Fruit Bowl,Salad,Soup


In [13]:
# Save as JSON
report_filename = "final_diet_plan.json"
with open(report_filename, "w") as f:
    # We save the dictionary we created earlier
    json.dump(final_report, f, indent=4)

# Save the 7-day table as a CSV
weekly_df.to_csv("weekly_diet_chart.csv", index=False)

print(f"Success! Files saved:")
print(f"1. {report_filename} (Detailed Report)")
print(f"2. weekly_diet_chart.csv (7-Day Schedule)")

Success! Files saved:
1. final_diet_plan.json (Detailed Report)
2. weekly_diet_chart.csv (7-Day Schedule)
